# Data preparation steps

90 % of the data was used for training for two reasons:

    - reducing data does not usually affect the topic model quality, so this reduces the computational load
    - having a test set would enable comparing model predictions after model training (not done yet but possible in the future)


In [6]:
import pandas as pd
import os

import numpy as np
import glob
import random
import re

In [21]:
# how the data was originally split into train and test set:

# load csv data
mdfpath= "../data/s24_metsa_actors_all.csv" # contains metadata sorted into separate columns from the "info" column
mdf=pd.read_csv(mdfpath, sep="\t")
print("length of the mdf dataset;", len(mdf), "\n")

fpath=fpath="../data/s24_metsa_actors_new_text2len.csv" 
# basically the same dataset, but the data set length differs slightly.. 
# so the two datasets are merged to work with a dataset with full metadata info
df=pd.read_csv(fpath, sep="\t")

# clean metadata

def clean_metadata(meta):
    
    textlist = meta.split()
    
    id_dict={"date":"","id":"","comment_id":"","time":"", "parent_id":"","author":"", "msg_type":""}
    
    for i,t in enumerate(textlist):
        if "date=" in t:
            date= t.replace("date=","")
            id_dict["date"] = date
        elif t.startswith("time="):
            time=t.replace("time=","")
            id_dict["time"] =time
        elif t.startswith("id="):#
            c_id = t.replace("id=","")
            id_dict["id"] = c_id
        elif t.startswith("comment_id="):#
            com_id = t.replace("comment_id=","")
            id_dict["comment_id"] = com_id
        elif "author=" in t:
            
            author= t.replace("author=","")
            if "parent_comment_id=" not in textlist[i+1] :
                author = author + " "+textlist[i+1]
            id_dict["author"] =author
        elif "parent_comment_id=" in t:
            p_id = t.replace("parent_comment_id=" ,"")
            id_dict["parent_id"] = p_id
        elif "msg_type=" in t:
            type_id = t.replace("msg_type=" ,"")
            id_dict["msg_type"] = type_id
    return  id_dict

cleaned_meta = df['info'].apply(clean_metadata)
print("Length of the df dataset")
print(len(cleaned_meta), len(df))
print(cleaned_meta[0],"\n")

metadatadf = pd.json_normalize(cleaned_meta)
df2= pd.concat([df.reset_index(),metadatadf], axis=1, join="inner")

df2["corpus_id"] =df2.index
df2["year"] =pd.DatetimeIndex(df2['date']).year
df2.sort_values(by="date",ascending=True, inplace=True)


# take files only from 2014 onwards
df2=df2[(df2["year"]>2013)&(df2["length"]>5)]
documents=list(df2["text"].drop_duplicates())
print("Got", len(documents), "documents. \n")

my_seed=99

# take files only from 2014 onwards
df2=df2[(df2["year"]>2013)&(df2["length"]>5)].drop_duplicates(subset=["text"])

train=df2.sample(frac=0.9,random_state=my_seed)
test=df2.drop(train.index)

#trainpath=fpath.replace(".csv","_train.csv")
#testpath=fpath.replace(".csv","_test.csv")

#train.to_csv(trainpath, sep="\t")
#test.to_csv(testpath, sep="\t")

documents=list(train["text"])
lemmas = list(train["lemmas"])

print("Training on", len(documents), "documents. \n")
print(documents[:5])

length of the mdf dataset; 50273 

Length of the df dataset
50783 50783
{'date': '2013-01-03', 'id': '11159573:59203728', 'comment_id': '59203728', 'time': '13:02:01', 'parent_id': '0', 'author': 'päästä', 'msg_type': 'comment'} 

Got 34040 documents. 

Training on 30636 documents. 

['Hei,minulla taas on aivan toisenlaisia kokemuksia.Ostin laiturin kyseiseltä filmalta noin kymmenen vuotta sitten ja vielläkin hyvä palvelu pelaa esim. laituri nostot ja laskut joka vuosi ja aina ajallaan.On aivan ihanaa kun keväällä aloitamme mökkeilyn on laituri jo odottamassa meitä,eikä talvisin ole ikinä tarvinnut pelätä että laituri rikkoutuisi,sillä se on aina nostettu turvaan rantaa,mökkimme kun on erittäin tuulisella paikalla.Hyvin on laituri palvellut meitä kaikki nämä vuodet ja kestänyt hyvin,en voi muuta kuin suositella.', 'Osta sitten metsää jos se noin hyvältä bisnekseltä tuntuu! Metsäteollisuus on kuitenkin maallemme perin tärkeää koska sen tuotteilla on varsin vakaa menekki ja jostainhan si

In [11]:
# training data
trainpath = "../data/s24_metsa_actors_new_text2len_train.csv"
print(trainpath)
train = pd.read_csv(trainpath, sep="\t", index_col=0)
display(train.head())
documents=list(train["text"].drop_duplicates())

# test set
testpath =  "../data/s24_metsa_actors_new_text2len_test.csv"
print(testpath)
test = pd.read_csv(testpath, sep="\t", index_col=0)
#test.head()

../data/s24_metsa_actors_new_text2len_train.csv


,index,text,info,lemmas,text_preprocessed,bigrams,length,date,id,comment_id,time,parent_id,author,msg_type,corpus_id,year
27782,27782,"Hei,minulla taas on aivan toisenlaisia kokemuk...",###C: <text comment_id=67417934 date=2014-04-2...,"hei , minä taas olla aivan toisenlainen kokemu...",hei taas aivan toisenlainen kokemuksias laitur...,hei taas aivan toisenlainen kokemuksias laitur...,53,2014-04-22,10731943:67417934,67417934,21:34:16,0,Suosittelen,comment,27782,2014
22833,22833,Osta sitten metsää jos se noin hyvältä bisneks...,###C: <text msg_type=comment thread_id=1648026...,ostaa sitten metsä jos se noin hyvä bisnes tun...,ostaa sitten metsä hyvä bisnes tuntua metsäteo...,ostaa sitten metsä hyvä bisnes tuntua metsäteo...,31,2020-08-05,16480264:103604378,103604378,19:26:18,103320837,Extyökkäri author_logged_in=y,comment,22833,2020
24039,24039,Karpohan paljasti myös vihreiden todellisen lu...,###C: <text msg_type=comment thread_id=1656720...,karpohan paljastaa myös vihre todellinen luonn...,karpohan paljastaa myös vihre todellinen luonn...,karpohan paljastaa myös vihre todellinen luonn...,27,2020-09-18,16567207:104219119,104219119,17:03:30,0,Anonyymi author_logged_in=n,comment,24039,2020
20566,20566,Finsilva oy:n omistajia on jyväjemmarien Metsä...,###C: <text msg_type=comment thread_id=1625320...,Finsilva oy omistaja olla jyvä#jemmari metsä#l...,finsilva oy omistaja jyväjemmari metsäliitto o...,finsilva oy omistaja jyväjemmari metsäliitto o...,16,2020-03-02,16253202:101242542,101242542,22:36:54,101094371,Anonyymi author_logged_in=n,comment,20566,2020
44835,44835,On jäänyt uhmaikä ja teinivinkeetkin välistä; ...,###C: <text msg_type=comment thread_id=1591018...,olla jäädä uhmaikä ja teini#vine välistä ; sam...,jäädä uhmaikä teinivine välistä samoin kaikki ...,jäädä uhmaikä teinivine välistä samoin kaikki ...,50,2019-08-17,15910188:98673334,98673334,22:19:32,98670041,M184 author_logged_in=y,comment,44835,2019


../data/s24_metsa_actors_new_text2len_test.csv


In [10]:
corpus_texts=list(train["text_preprocessed"])
print(len(corpus_texts))
print(len([t for t in corpus_texts if len(t.split()) > 5])) # just to make sure that tre training set does not contain short documents

30636
30636
